In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

In [3]:
import tensorflow_datasets as tfds


In [4]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [5]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [6]:
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image,label):
    image = tf.cast(image, tf.float32)
    image/= 255.
    return image,label

scaled_trained_and_validation = mnist_train.map(scale)

In [7]:
test_data = mnist_test.map(scale)

In [8]:
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_trained_and_validation.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data= validation_data.batch(num_validation_samples)
test_data= test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


In [9]:
input_size=784
output_size=10
hidden_layer_size=100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
    
                            ])

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.3312 - accuracy: 0.9054 - val_loss: 0.1605 - val_accuracy: 0.9535 - 3s/epoch - 6ms/step
Epoch 2/5
540/540 - 1s - loss: 0.1368 - accuracy: 0.9592 - val_loss: 0.1059 - val_accuracy: 0.9677 - 1s/epoch - 3ms/step
Epoch 3/5
540/540 - 1s - loss: 0.0956 - accuracy: 0.9711 - val_loss: 0.0827 - val_accuracy: 0.9755 - 1s/epoch - 3ms/step
Epoch 4/5
540/540 - 1s - loss: 0.0747 - accuracy: 0.9775 - val_loss: 0.0681 - val_accuracy: 0.9797 - 1s/epoch - 3ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0585 - accuracy: 0.9821 - val_loss: 0.0666 - val_accuracy: 0.9795 - 1s/epoch - 3ms/step


In [12]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 310ms/step - loss: 0.0953 - accuracy: 0.9703


## Predicting model using real image

In [13]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

image_path="4.png"
im = image.load_img(image_path, target_size=(28, 28), color_mode = "grayscale")
img = image.img_to_array(im)           #returns a 3D Numpy array with pixel and color values  [height, width, channels] 
img = tf.expand_dims(img, axis=0)      #changes the shape by adding 1 more dimension. DEEP LEARNING EXPECTS BATCH OF IMAGES
                                       #expected image shape [batch, height, width, channels] || channeel is no of colors
                                       #For example, a vector with 10 elements could be treated as a 10x1 matrix.
                                       #if vector (10,) ; axis = 0 > (1,10) ; axis = -1 > (10,1) 
img = scale(img, '')[0]  #scaling image using scale func (dividing by float 255) | [0] cuz func return 2 values image,label

# model.predict_classes is deprecated

# model.predict(img)[0]                # this will show the probabilities of each class from class 0 to class 9

num = np.argmax(model.predict(img)[0], axis=-1)  # this will give you the index of class which has highest probability

if(num==4):
    print("This is number FOUR")

This is number FOUR


In [14]:
im

In [15]:
image_path="7.png"
im = image.load_img(image_path, target_size=(28, 28), color_mode = "grayscale")
img = image.img_to_array(im)
img = tf.expand_dims(img, axis=0)      
img = scale(img, '')[0]
num = np.argmax(model.predict(img)[0], axis=-1)

In [16]:
im

In [17]:
num

7

In [19]:
image_path="1.png"
im = image.load_img(image_path, target_size=(28, 28), color_mode = "grayscale")
img = image.img_to_array(im)
img = tf.expand_dims(img, axis=0)      
img = scale(img, '')[0]
np.argmax(model.predict(img)[0], axis=-1)

1